### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

/home/randomwalk10/anaconda3/envs/aind/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/randomwalk10/anaconda3/envs/aind/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [58]:
user_movie_subset

movie_id,73486,75314,68646,99685
user_id,,,,
265,10.0,10.0,10.0,10.0
1023,10.0,4.0,9.0,10.0
1683,8.0,9.0,10.0,5.0
6571,9.0,8.0,10.0,10.0
11639,10.0,5.0,9.0,9.0
13006,6.0,4.0,10.0,6.0
14076,9.0,8.0,10.0,9.0
14725,10.0,5.0,9.0,8.0
23548,7.0,8.0,10.0,8.0


In [43]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = n_users*n_movies-np.sum(np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for i in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for r in range(n_users):
            for c in range(n_movies):
                # if the rating exists
                if not np.isnan(ratings_mat.item(r, c)):    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    err = ratings_mat.item(r,c) - np.dot(user_mat[r, :], movie_mat[:, c])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += err**2
                    # update the values in each matrix in the direction of the gradient
                    old_user_row = user_mat[r, :]
                    user_mat[r, :] += 2*err*learning_rate*movie_mat[:, c]
                    movie_mat[:, c] += 2*err*learning_rate*old_user_row
        # print results for iteration
        print("#%d sse_accum is %f and delta is %f" % (i+1, sse_accum, sse_accum-old_sse))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [51]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
#1 sse_accum is 3189.714530 and delta is 3189.714530
#2 sse_accum is 1003.442285 and delta is -2186.272244
#3 sse_accum is 297.053296 and delta is -706.388989
#4 sse_accum is 224.981003 and delta is -72.072293
#5 sse_accum is 216.828876 and delta is -8.152127
#6 sse_accum is 213.248055 and delta is -3.580821
#7 sse_accum is 210.099690 and delta is -3.148365
#8 sse_accum is 206.703478 and delta is -3.396212
#9 sse_accum is 202.901036 and delta is -3.802442
#10 sse_accum is 198.640097 and delta is -4.260938


In [52]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.0844403   8.78763149 10.78282824 10.04418275]
 [ 8.6112317   7.38929084  9.36531398  8.60249284]
 [ 7.82166888  6.75007573  8.25812776  7.86864034]
 [ 9.54937019  8.32999045 10.03396714  9.5549262 ]
 [ 8.3948008   7.11927623  9.46954688  8.36444964]
 [ 6.68045882  5.61149036  7.92460052  6.5738904 ]
 [ 9.25546842  8.10878076  9.50392909  9.28392843]
 [ 8.18479475  7.09378536  8.66282812  8.21013326]
 [ 8.50586866  7.41066219  8.85847424  8.52718736]
 [ 7.74753806  6.84376602  7.83983189  7.76552392]
 [ 8.9406097   7.87959103  9.10617482  8.96538947]
 [ 9.67534613  8.65674278  9.78615671  9.62612606]
 [ 9.36068342  8.27656924  9.4015917   9.39882102]
 [ 6.81059628  6.13410897  6.27542265  6.91417463]
 [ 9.62045883  8.25503883 10.67105261  9.5676846 ]
 [ 9.7588335   8.56088416 10.1854373   9.74753842]
 [ 8.37793929  7.4602953   8.39539936  8.38395904]
 [ 7.24671179  6.52635523  6.79857939  7.32797494]
 [ 8.32373764  7.20482674  9.08638371  8.26913081]
 [ 8.07145811  6.90530496  8.74

**Write your summary here.**

The result of FunkSVD is an approximation of the original subset of data.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [59]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
#1 sse_accum is 3689.893896 and delta is 3689.893896
#2 sse_accum is 1351.478882 and delta is -2338.415014
#3 sse_accum is 350.219625 and delta is -1001.259257
#4 sse_accum is 219.829410 and delta is -130.390215
#5 sse_accum is 207.351339 and delta is -12.478071
#6 sse_accum is 203.121877 and delta is -4.229461
#7 sse_accum is 199.424876 and delta is -3.697001
#8 sse_accum is 195.250005 and delta is -4.174872
#9 sse_accum is 190.406456 and delta is -4.843549
#10 sse_accum is 184.854106 and delta is -5.552350
#11 sse_accum is 178.590008 and delta is -6.264098
#12 sse_accum is 171.640163 and delta is -6.949845
#13 sse_accum is 164.067603 and delta is -7.572561
#14 sse_accum is 155.978435 and delta is -8.089168
#15 sse_accum is 147.521332 and delta is -8.457103
#16 sse_accum is 138.878801 and delta is -8.642530
#17 sse_accum is 130.250419 and delta is -8.628382
#18 sse_accum is 121.830424 and delta is -8.419996
#19 sse_accum is 113.

#233 sse_accum is 0.000000 and delta is -0.000000
#234 sse_accum is 0.000000 and delta is -0.000000
#235 sse_accum is 0.000000 and delta is -0.000000
#236 sse_accum is 0.000000 and delta is -0.000000
#237 sse_accum is 0.000000 and delta is -0.000000
#238 sse_accum is 0.000000 and delta is -0.000000
#239 sse_accum is 0.000000 and delta is -0.000000
#240 sse_accum is 0.000000 and delta is -0.000000
#241 sse_accum is 0.000000 and delta is -0.000000
#242 sse_accum is 0.000000 and delta is -0.000000
#243 sse_accum is 0.000000 and delta is -0.000000
#244 sse_accum is 0.000000 and delta is -0.000000
#245 sse_accum is 0.000000 and delta is -0.000000
#246 sse_accum is 0.000000 and delta is -0.000000
#247 sse_accum is 0.000000 and delta is -0.000000
#248 sse_accum is 0.000000 and delta is -0.000000
#249 sse_accum is 0.000000 and delta is -0.000000
#250 sse_accum is 0.000000 and delta is -0.000000


In [60]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999488  9.99999703  9.99999882 10.00000545]
 [ 9.99999733  3.9999985   8.99999944 10.00000287]
 [ 8.00000052  9.00000039 10.00000022  4.99999947]
 [ 9.00000409  8.00000242 10.00000102  9.99999573]
 [ 9.99999498  4.99999714  8.9999989   9.00000534]
 [ 5.99999879  3.99999936  9.9999998   6.00000132]
 [ 9.00000128  8.0000008  10.00000039  8.99999872]
 [10.00000374  5.00000222  9.00000094  7.9999961 ]
 [ 6.99999855  7.99999916  9.99999968  8.00000158]
 [ 9.00000401  5.0000023   9.00000093  6.99999581]
 [ 8.99999868  7.99999921  9.99999968  8.00000143]
 [ 9.00000309 10.00000175 10.0000007   8.99999678]
 [ 9.99999297  8.99999592  9.99999836  8.00000745]
 [ 4.9999996   7.99999981  4.99999996  8.00000046]
 [ 9.99999993  8.         10.00000006 10.00000014]
 [ 9.00000193  9.00000116 10.00000051  9.99999801]
 [ 8.99999682  7.99999822  7.99999932  8.00000338]
 [10.00000162  8.00000097  1.00000044  9.99999835]
 [ 4.99999909  5.9999995   9.99999983 10.00000099]
 [ 8.00000542  7.00000316 10.00

**Write your summary here.**

The approximation and the original data are almost identical.

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [61]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [62]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
#1 sse_accum is 3518.594526 and delta is 3518.594526
#2 sse_accum is 1251.120644 and delta is -2267.473882
#3 sse_accum is 339.895532 and delta is -911.225112
#4 sse_accum is 218.976377 and delta is -120.919154
#5 sse_accum is 204.375360 and delta is -14.601018
#6 sse_accum is 197.881508 and delta is -6.493851
#7 sse_accum is 192.200657 and delta is -5.680851
#8 sse_accum is 186.266714 and delta is -5.933943
#9 sse_accum is 179.771813 and delta is -6.494901
#10 sse_accum is 172.593993 and delta is -7.177820
#11 sse_accum is 164.683366 and delta is -7.910627
#12 sse_accum is 156.039668 and delta is -8.643698
#13 sse_accum is 146.711333 and delta is -9.328334
#14 sse_accum is 136.797701 and delta is -9.913633
#15 sse_accum is 126.447770 and delta is -10.349931
#16 sse_accum is 115.852468 and delta is -10.595302
#17 sse_accum is 105.229573 and delta is -10.622895
#18 sse_accum is 94.802799 and delta is -10.426774
#19 sse_accum is 84

#243 sse_accum is 0.000000 and delta is -0.000000
#244 sse_accum is 0.000000 and delta is -0.000000
#245 sse_accum is 0.000000 and delta is -0.000000
#246 sse_accum is 0.000000 and delta is -0.000000
#247 sse_accum is 0.000000 and delta is -0.000000
#248 sse_accum is 0.000000 and delta is -0.000000
#249 sse_accum is 0.000000 and delta is -0.000000
#250 sse_accum is 0.000000 and delta is -0.000000


In [63]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.278914836516336:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [64]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, 4, 0.005, 20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
#1 sse_accum is 251896.739393 and delta is 251896.739393
#2 sse_accum is 115547.500455 and delta is -136349.238939
#3 sse_accum is 79284.992192 and delta is -36262.508262
#4 sse_accum is 61242.172611 and delta is -18042.819582
#5 sse_accum is 49885.259158 and delta is -11356.913452
#6 sse_accum is 41945.153802 and delta is -7940.105357
#7 sse_accum is 36040.025299 and delta is -5905.128502
#8 sse_accum is 31466.947899 and delta is -4573.077400
#9 sse_accum is 27824.544747 and delta is -3642.403152
#10 sse_accum is 24864.248622 and delta is -2960.296125
#11 sse_accum is 22421.700978 and delta is -2442.547644
#12 sse_accum is 20382.118782 and delta is -2039.582197
#13 sse_accum is 18661.998051 and delta is -1720.120731
#14 sse_accum is 17198.951593 and delta is -1463.046458
#15 sse_accum is 15945.531321 and delta is -1253.420272
#16 sse_accum is 14865.082468 and delta is -1080.448854
#17 sse_accum is 13928.798023 and delta is -936.

`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [71]:
# Replace each of the comments below with the correct values
num_user = first_1000_users.shape[0]
num_movie = first_1000_users.shape[1]
num_ratings = num_user*num_movie-np.sum(np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.sum(np.isnan(first_1000_users)) # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [72]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?


In [74]:
preds

array([[3.25535203, 5.11790583, 3.54533404, ..., 5.7805731 , 5.89873826,
        3.92631154],
       [3.8846001 , 3.87180992, 3.41021228, ..., 6.00371102, 6.09892903,
        4.60339827],
       [3.13483773, 4.01359303, 3.14207777, ..., 5.12571378, 5.22751267,
        3.62920951],
       ...,
       [3.07051418, 3.33333189, 2.80634088, ..., 4.83107934, 4.95007445,
        3.69205198],
       [3.86783205, 4.84624249, 3.60221708, ..., 6.48843803, 6.82044688,
        5.30911188],
       [2.90571153, 3.03412032, 2.69359235, ..., 4.45339072, 4.48558944,
        3.18969812]])